In [2]:
import pandas as pd
from pearlsim.ml_utilities import *
import os
import random
import numpy as np

In [2]:
current_data = pd.read_csv("training_data/current_target.csv", index_col=0)

In [3]:
file_names = os.listdir("burnup_model_concentrations/")
concentrations = []
for name in file_names:
    if "bumat0" in name:
        concentrations += extract_from_bumat("burnup_model_concentrations/"+name)
print(f"{len(concentrations)} material definitions loaded.")

64640 material definitions loaded.


,current_E1,current_E2,current_E3,current_E4,current_E5,current_E6,current_E7,current_E8,current_E9,current_E10,current_E11,current_E12,current_E13,current_E14,current_E15,current_E16,current_E17,current_E18
0,7.521810e+13,2.296940e+14,2.601430e+14,1.517750e+14,4.576870e+13,1.843990e+13,3.207780e+13,4.530800e+13,8.276560e+13,4.247150e+13,2.226990e+14,4.202590e+14,7.176590e+13,3.060270e+14,1.511140e+14,7.651530e+13,1.690220e+14,6.137510e+13
1,1.288390e+14,2.957060e+14,3.035910e+14,1.506420e+14,3.927580e+13,6.221970e+12,3.759940e+13,5.069570e+13,7.171990e+13,2.718650e+13,1.761660e+14,3.414380e+14,4.359710e+13,2.252180e+14,1.351310e+14,4.964670e+13,1.274430e+14,3.622090e+13
2,1.574750e+14,3.491190e+14,4.308930e+14,2.516200e+14,7.637080e+13,2.070090e+13,5.665870e+13,8.169960e+13,9.772030e+13,5.834760e+13,3.326930e+14,5.704750e+14,9.273520e+13,4.308400e+14,2.584410e+14,9.482760e+13,1.957230e+14,6.621530e+13
3,9.289470e+13,2.882190e+14,3.328780e+14,2.535090e+14,1.063370e+14,2.030070e+13,7.501210e+13,1.306780e+14,1.330360e+14,8.866810e+13,4.853850e+14,9.425920e+14,1.203470e+14,6.170240e+14,2.759670e+14,1.301450e+14,2.353390e+14,9.656350e+13
4,1.458510e+14,3.336470e+14,3.548030e+14,2.126570e+14,6.514330e+13,1.657510e+13,5.808070e+13,8.472370e+13,9.968180e+13,5.483560e+13,2.864000e+14,5.685470e+14,7.786030e+13,4.321820e+14,2.330110e+14,9.696060e+13,2.119090e+14,7.406250e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43995,1.612400e+15,3.833830e+15,3.608670e+15,1.852630e+15,3.063580e+14,7.938810e+13,2.693710e+14,3.342130e+14,4.339780e+14,2.524770e+14,1.185790e+15,2.361930e+15,3.239130e+14,1.526050e+15,8.136170e+14,3.487110e+14,6.467360e+14,2.320140e+14
43996,1.574190e+15,3.510330e+15,3.531940e+15,1.745730e+15,3.548930e+14,9.333640e+13,2.727840e+14,4.071240e+14,4.770560e+14,2.985580e+14,1.387620e+15,2.782090e+15,4.493270e+14,1.996390e+15,1.006660e+15,4.347560e+14,7.474500e+14,3.048320e+14
43997,3.302340e+14,8.090940e+14,8.247460e+14,3.493140e+14,3.249070e+13,1.541190e+13,2.879100e+13,5.518250e+13,7.422030e+13,3.471460e+13,1.522600e+14,2.814420e+14,3.797200e+13,1.937700e+14,1.054080e+14,3.249750e+13,6.249970e+13,1.898940e+13
43998,1.310400e+15,2.987010e+15,2.903520e+15,1.393020e+15,1.553830e+14,6.070740e+13,1.662380e+14,2.405380e+14,2.561350e+14,1.736680e+14,7.692230e+14,1.380810e+15,1.847360e+14,9.453930e+14,4.614180e+14,2.624660e+14,4.620700e+14,1.764480e+14


In [29]:
def generate_pebble_burnup_model(template_path, current_vector, concentrations, temp, time):
    lib_str = get_cross_section_string(temp)
    surface_current = current_vector.drop(labels=["power"])
    power = current_vector['power']
    energy_bins = ENERGY_BINS
    with open(template_path, "r") as f:
        input_s = f.read()
    concentration_s = ""
    for key in concentrations.keys():
        concentration_s  += f"  {key}    {concentrations[key]}\n".replace("<lib>", lib_str)
    input_s = input_s.replace("<concentrations>", concentration_s)
    input_s = input_s.replace("<temperature>", str(temp))
    input_s = input_s.replace("<time>", str(time))
    input_s = input_s.replace("<power>", str(power))

    weights = [0]+list((surface_current/1e9).astype(int))
    source_str = f"sb {len(weights)} 1\n"
    for i in range(len(surface_current)+1):
        source_str += f"  {energy_bins[i]} {weights[i]}\n"
    input_s = input_s.replace("<current>", source_str)
    return input_s

In [30]:
file_name = "burnup_models/burnup0.serpent"

input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current_data.iloc[10], concentrations[3], 959, 6.525)

with open(file_name, 'w') as f:
    f.write(input_s)

In [3]:
starting_iter = 1
iterations = 1000
np.random.seed(int(starting_iter*iterations*5234))
time_steps = [6.525]
series_list = []

In [60]:
for i in range(starting_iter,iterations+starting_iter+1):
    conc_dict = concentrations[np.random.randint(len(concentrations))]
    conc_series = pd.Series(conc_dict)
    current_series = current_data.iloc[np.random.randint(len(current_data))]
    days = time_steps[ np.random.randint(len(time_steps)) ]
    scalar_series = pd.Series( {"depletion_time": days} )
    step_series = pd.concat([conc_series, current_series, scalar_series])
    csv_file_name = f"burnup_models/input_series_{i}.csv"
    step_series.to_frame().T.to_csv(csv_file_name)
    series_list += [step_series]
    
    input_file_name = f"burnup_models/burnup{i}.serpent"
    input_s = generate_pebble_burnup_model("pebble_gFHR_template.serpent", current_series, conc_dict, 959,  days)

    with open(input_file_name, 'w') as f:
        f.write(input_s)

num_cores = 20
iterations = 1000
for i in range(1,iterations+1):

    file_name = f"burnup{i}.serpent"
    os.system(f"sss2_2_0 {file_name} -omp {num_cores}")

In [17]:
frames = []
concentration_output = []

for i in range(starting_iter,iterations+starting_iter+1):
    try:
        conc_dict = extract_from_bumat(f"burnup_models/burnup{i}.serpent.bumat1")
        series_data = pd.read_csv(f"burnup_models/input_series_{i}.csv", index_col=0)
        concentration_output += [pd.DataFrame(conc_dict)]
        frames += [series_data]
    except:
        print(f"Couldn't find data for step {i}, skipping.")

Couldn't find data for step 1001, skipping.


In [18]:
features_df = pd.concat(frames, sort=False).drop(columns=["depletion_time"]).fillna(0.0).reset_index(drop=True)
display(features_df.iloc[0:5])
print(features_df.shape)

,10010,10020,1003<lib>,20030,20040,3006<lib>,3007<lib>,40090,5010<lib>,5011<lib>,...,551280,952380,962380,451000,461000,561280,571340,581340,531220,541220
0,2.412802e-11,1.960346e-14,3.827746e-17,2.096405e-18,3.435394e-08,0.000000e+00,0.000000e+00,4.126630e-10,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.051315e-09,9.387650e-11,3.152308e-14,1.218747e-15,2.283355e-07,7.184654e-29,4.410228e-21,1.098716e-08,5.121017e-23,2.312141e-14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.571253e-14,4.407622e-18,1.038205e-25,1.372871e-28,1.879975e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.533491e-10,6.683680e-11,2.348143e-14,3.174550e-15,1.329958e-06,0.000000e+00,0.000000e+00,1.776475e-08,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.186991e-09,1.948494e-10,3.596826e-13,2.122895e-14,1.285585e-06,3.796617e-26,3.409452e-18,1.671971e-08,9.722919e-22,2.110547e-11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(1000, 1336)


In [20]:
post_burn_conc_df = pd.concat(concentration_output, sort=False).fillna(0.0).reset_index(drop=True)
display(post_burn_conc_df.iloc[0:5])
print(post_burn_conc_df.shape)

,10010,10020,1003<lib>,20030,20040,20060,3006<lib>,3007<lib>,30080,40090,...,812020,812030,812040,812050,822020,822030,82204<lib>,822050,82206<lib>,822071
0,3.170006e-10,5.203985e-12,4.257245e-15,5.530865e-18,0.000006,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.041544e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.831726e-09,1.169126e-10,3.449613e-14,1.253247e-15,0.000003,3.714596e-34,2.547251e-27,4.417136e-21,4.056320e-26,4.031333e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.161529e-09,1.769992e-11,2.222222e-12,5.670086e-16,0.000028,5.541271e-15,1.340879e-09,4.629674e-16,1.443390e-28,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.141791e-08,3.899725e-10,4.974759e-14,3.311051e-15,0.000016,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.533620e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.921634e-09,4.273413e-10,3.784859e-13,2.165465e-14,0.000012,1.371441e-30,9.106535e-24,3.464825e-18,1.744534e-22,1.864751e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(1000, 1455)


In [21]:
features_df.to_csv("training_data/burnup_features.csv")
post_burn_conc_df.to_csv("training_data/burnup_target.csv")